In [1]:
import pandas as pd
import numpy as np
import xpress as xp # FICO Xprerss Solver
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

from atcs import ATCS
import fig_plot as fp

In [2]:
N_sample = 10
seed = 1
solver_type = 'minlp'
model_type = 'deterministic'
sample_type = f'head_{N_sample}'
data = ATCS(seed = 1)
data.choose_subset_point(N_sample, randomized = False) # Choose subset data

/Users/thee/Trm_proj/repo/uoe-risk-and-logistics/assignment02/atcs.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.l_sub_df.drop(self.l_sub_df.index[4], axis = 0, inplace=True)


In [3]:
location_fig = fp.create_l_fig()
location_fig = fp.add_point_to_l_fig(location_fig, data.l_df, size=5, name='All Data Points', color='blue')
location_fig = fp.add_point_to_l_fig(location_fig, data.l_sub_df, size=5, name='Sub Data Points', color='red')
location_fig.show()

In [4]:
data.r_sub_df.head()

,range
0,127.868274
1,87.246488
2,64.393799
3,100.815352
4,36.021544


In [5]:
robot_loc = data.l_sub_df.to_numpy()

In [6]:
robot_loc

array([[-120.2408148 ,  -79.23837427],
       [-134.5378392 ,  -83.4262141 ],
       [-105.2822148 ,  -84.68170509],
       [-105.4532435 ,  -77.96400711],
       [-115.5387208 ,  -89.76382182],
       [-134.2914704 ,  -77.79880444],
       [-104.5018311 ,  -79.2271725 ],
       [-140.1312288 ,  -87.85896352],
       [-122.5076672 ,  -86.90936566]])

In [7]:
# MINLP Model
# Data
robot_id = data.r_sub_df.index.to_numpy()
robot_loc = data.l_sub_df.to_numpy()
robot_range = data.r_sub_df.to_numpy().flatten()
x_min,x_max = np.min(robot_loc, axis=0)[0], np.max(robot_loc, axis=0)[0]
y_min,y_max = np.min(robot_loc, axis=0)[1], np.max(robot_loc, axis=0)[1]
dist_matrix = data.get_distance_matrix(sample_subset = True)

# Creates an empty Model
model = xp.problem(name='MINLP_Deterministic')

# Given Inputs and Parameters
R = data.r_sub_df['range'].to_dict()
L_Range = data.r_min # km
U_Range = data.r_max # km
U_Robot = data.q # Max robots per charger
U_Charger = data.m # Max chargers per station
L_Station = int(np.ceil(N_sample/(U_Charger * U_Robot))) # Maximum Robots per opened station
Dist_Max = np.sqrt((x_max - x_min)**2 + (y_max - y_min)**2)
C_b = data.c_b # Investment cost per station
C_h = data.c_h # Cost of moving a robot
C_m = data.c_m # Maintenance cost per charger
C_c = data.c_c # Charging cost per km


Long_V, Lat_V = data.l_sub_df['longitude'].to_dict(), data.l_sub_df['latitude'].to_dict()

# Parameters
# U_Station = L_Station + 1 # For now
U_Station = len(robot_id)
data.set_output_folder(solver_type, model_type, name = f'{N_sample}_{sample_type}samp_{U_Station}s_seed{seed}')

# U_Station = N_sample
# Sets and Notation
V = data.r_sub_df.index.to_numpy() # Robot Vehicle sets
S = np.arange(U_Station)


# Output Variables
x = model.addVariables(S, name = 'x', vartype = xp.continuous, lb = x_min, ub = x_max) # Longitude of station s
y = model.addVariables(S, name = 'y', vartype = xp.continuous, lb = y_min, ub = y_max) # Latitude of station s

eta = model.addVariables(S, name = 'eta', vartype = xp.integer) # Number of charger installed at station s
mu = model.addVariables(S, name = 'mu', vartype = xp.binary) # Whether station s is built
beta = model.addVariables(V, S, name = 'beta', vartype = xp.binary) # Whether robot v is covered by station s
alpha = model.addVariables(V, S, name = 'alpha', vartype = xp.binary) # Whether robot v is brought to station s by taking penalty
d = model.addVariables(V, S, name = 'd', vartype = xp.continuous, lb = 0) # Distance from v to s


# Constraints
# Robot v can be allocated to charging station s iff it is built
model.addConstraint(beta[v,s] <= mu[s] for v in V for s in S) 

# Each Robot v must be allocated to only one station
model.addConstraint(xp.Sum(beta[v,s] for s in S) == 1 for v in V)

# Number of chargers per station
model.addConstraint(eta[s] >= xp.Sum(beta[v,s] for v in V)/U_Robot for s in S)
model.addConstraint(eta[s] <= U_Charger for s in S)
# Robot v can be brought to station s by human iff the robot is allocated to that station
model.addConstraint(alpha[v,s] <= beta[v,s] for v in V for s in S)

# Distance from robot v to station s
# If Robot v is allocated to station s,
#       If its range can reach s, alpha = 0
#       Else; distance is 0 and it is brought to s by taking penalty (alpha = 1)
# Else; d[v,s] = 0
model.addConstraint(d[v,s] <= R[v] * (1 - alpha[v,s]) for v in V for s in S)
model.addConstraint(d[v,s] >= xp.sqrt((Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2)
                                - Dist_Max * (alpha[v,s] + (1 - beta[v,s])) 
                                        for v in V for s in S)
# model.addConstraint(d[v,s]**2 >= (Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2
#                                 - (np.max(dist_matrix)**2) * (alpha[v,s] + (1 - beta[v,s])) 
#                                         for v in V for s in S)
# model.addConstraint(d[v,s] >= (Long_V[v] - x[s])**2 + (Lat_V[v] - y[s])**2
#                                 - np.max(dist_matrix) * (alpha[v,s] + (1 - beta[v,s])) 
#                                         for v in V for s in S)
model.addConstraint(d[v,s] <= Dist_Max * beta[v,s] for v in V for s in S)
# model.addConstraint(alpha[120,0] == 1)
# model.addConstraint(d[v,s] <= np.min(dist_matrix[dist_matrix != 0]) * (1 - alpha[v,s]) for v in V for s in S)
# model.addConstraint(d[v,s] >= np.min(dist_matrix[dist_matrix != 0]) - np.max(dist_matrix) * (alpha[v,s] + (1 - beta[v,s])) 
#                                     for v in V for s in S)
# model.addConstraint(d[v,s] <= np.max(dist_matrix)*beta[v,s] for v in V for s in S)

# Objective Function
Build_station_cost = xp.Sum(C_b * mu[s] for s in S)
Build_charger_cost = xp.Sum(C_m * eta[s] for s in S)
Penalty_cost = xp.Sum(C_h * alpha[v,s] for v in V for s in S)
Charging_cost = xp.Sum(C_c * (beta[v,s]*U_Range - (beta[v,s]*R[v] - d[v,s])) for v in V for s in S)

obj = Build_station_cost + Build_charger_cost + Charging_cost + Penalty_cost

model.setObjective(obj, sense=xp.minimize)
# model.setControl('miprelstop', 1e-3)
model.setControl('maxtime', 3600)
# Solve the problem
model.solve()

Original problem size
   linear:    432 rows, 279 columns, 909 linear coefficients
   nonlinear: 81 coefficients, 2511 tokens
             567 mul             0 div        81 sqrt
Nonlinear presolve
   linear presolve removed 90 rows, 0 columns, 90 linear coefficients
   creating 162 '+' clusters removed 162 tokens
Presolved problem size
   linear:    342 rows, 280 columns, 819 linear coefficients
   nonlinear: 81 coefficients, 2349 tokens
             405 mul             0 div        81 sqrt
               0 exp             0 log       162 pow
Problem is nonlinear presolved
FICO Xpress v9.4.2, Hyper, solve started 17:01:00, Apr 4, 2025
Control settings used:
MAXTIME = 3600
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Maximum expanded nl-formula size: 29  (row 'R271')
Total tokens: 2349
  11  parallel calculation threads
  Jacobian: symbolic differentiation
          81 base AD formula, 24 average complexity
         324 in the Jacobian, 14 average complexi

  1 O  7627.2561  7627.25      .00      .00     1.00           164    0        0
  2 O  7627.2561               .00      .00                     19    0        0
  3 O  7627.2561  7627.25      .00      .00      .00      .00    0    0 *      0
Best solution achieved by cascading at iteration 3
Final linearization will not match solution returned
  Restored solution with objective:  7627.2561

Xpress-SLP stopped after 3 iterations. 19 unconverged items
Problem solved using Xpress-NLP SLP
No unconverged values in active constraints
FICO Xpress v9.4.2, Hyper, solve started 17:01:00, Apr 4, 2025
Heap usage: 372KB (peak 392KB, 686KB system)
Minimizing MILP MINLP_Deterministic using 1 thread and up to 18GB memory
Original problem has:
       523 rows          542 cols         1738 elements       180 entities

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e-05,  3.76e+01] / [ 1.60e-04,  3.52e+01]
  RHS and bounds [min,max] : [ 1

(<SolveStatus.COMPLETED: 3>, <SolStatus.FEASIBLE: 2>)

In [8]:
np.min(dist_matrix[dist_matrix != 0])

1.5813830520679033

In [9]:
model.getSolution(obj)

7924.761117096377

In [10]:
mu_dict = model.getSolution(mu)
eta_dict = model.getSolution(eta)
x_dict = model.getSolution(x)
y_dict = model.getSolution(y)

station_df = pd.DataFrame([mu_dict, eta_dict, x_dict, y_dict], ['mu','eta','longitude','latitude']).T
station_df['Building_Station_Cost'] = station_df['mu'] * C_b
station_df['Building_Charger_Cost'] = station_df['eta'] * C_m

In [11]:
beta_dict = model.getSolution(beta)
alpha_dict = model.getSolution(alpha)
d_dict = model.getSolution(d)

robot_df = pd.DataFrame([beta_dict, alpha_dict, d_dict],['beta','alpha','d']).T.reset_index()
robot_df.rename(columns={'index':'(v,s)'}, inplace=True)
robot_df['v'], robot_df['s'] = robot_df['(v,s)'].apply(lambda x: x[0]), robot_df['(v,s)'].apply(lambda x: x[1])
robot_df.drop(['(v,s)'], axis = 1, inplace= True)
robot_df = robot_df[['v','s','beta','alpha','d']]
# robot_df = robot_df.groupby(['s','v']).sum()

In [12]:
model.getSolution(Charging_cost)

424.76111709637877

In [13]:
robot_df[robot_df['beta']==1]

,v,s,beta,alpha,d
4,0,4,1.0,0.0,4.112801
13,1,4,1.0,0.0,14.404893
22,2,4,1.0,0.0,14.910527
31,3,4,1.0,0.0,15.636673
40,4,4,1.0,0.0,7.889785
49,5,4,1.0,0.0,15.207611
58,6,4,1.0,0.0,16.165809
67,7,4,1.0,0.0,20.500147
76,8,4,1.0,0.0,4.278882


In [14]:
station_df

,mu,eta,longitude,latitude,Building_Station_Cost,Building_Charger_Cost
0,0.0,0.0,-140.131229,-89.763822,0.0,0.0
1,0.0,0.0,-140.131229,-89.763822,0.0,0.0
2,0.0,0.0,-140.131229,-89.763822,0.0,0.0
3,0.0,0.0,-140.131229,-89.763822,0.0,0.0
4,1.0,5.0,-120.133140,-83.349774,5000.0,2500.0
5,0.0,0.0,-140.131229,-89.763822,0.0,0.0
6,0.0,0.0,-140.131229,-89.763822,0.0,0.0
7,0.0,0.0,-140.131229,-89.763822,0.0,0.0
8,0.0,0.0,-140.131229,-89.763822,0.0,0.0


In [15]:
robot_df.to_csv('minlp_output/deterministic/20_samp_20s_seed1/robot_df.csv')
station_df.to_csv('minlp_output/deterministic/20_samp_20s_seed1/station_df.csv')

In [16]:
asdfasdf

NameError: name 'asdfasdf' is not defined

In [17]:
heu_station_df = pd.DataFrame( [(-118.57813985619894, -83.21073254829084)])
heu_station_df.columns = ['longitude','latitude']

In [18]:
robot_df_out = robot_df[robot_df['beta']==1]

In [19]:
heu_dis_df = data.l_sub_df.copy()
heu_dis_df['d'] = np.sqrt((data.l_sub_df['longitude'] - heu_station_df['longitude'].values[0])**2 + (data.l_sub_df['latitude'] - heu_station_df['latitude'].values[0])**2)
robot_df_out['d_heu'] = heu_dis_df['d'].to_list()


/var/folders/hx/pk7ddg2n1sn4xbjb6f8ks7r40000gn/T/ipykernel_38363/1605398997.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
robot_df_out

,v,s,beta,alpha,d,d_heu
4,0,4,1.0,0.0,4.112801,4.306288
13,1,4,1.0,0.0,14.404893,15.961154
22,2,4,1.0,0.0,14.910527,13.377047
31,3,4,1.0,0.0,15.636673,14.134746
40,4,4,1.0,0.0,7.889785,7.223645
49,5,4,1.0,0.0,15.207611,16.619197
58,6,4,1.0,0.0,16.165809,14.629122
67,7,4,1.0,0.0,20.500147,22.048621
76,8,4,1.0,0.0,4.278882,5.396394


In [21]:
heu_station_df

,longitude,latitude
0,-118.57814,-83.210733


In [22]:
location_fig = fp.create_l_fig()
# location_fig = fp.add_point_to_l_fig(location_fig, data.l_df, size=5, name='All Data Points', color='blue')
location_fig = fp.add_point_to_l_fig(location_fig, data.l_sub_df, size=5, name='Sub Data Points', color='red')
location_fig = fp.add_point_to_l_fig(location_fig, station_df[station_df['mu'] != 0], size=7, name='MINLP Stations', color='green')
location_fig = fp.add_point_to_l_fig(location_fig, heu_station_df, size=7, name='Heuristic Stations', color='Brown')
location_fig.show()

In [ ]:
data.l_sub_df

,longitude,latitude
0,-120.240815,-79.238374
1,-134.537839,-83.426214
2,-105.282215,-84.681705
3,-105.453243,-77.964007
4,-115.538721,-89.763822
5,-134.291470,-77.798804
6,-104.501831,-79.227172
7,-140.131229,-87.858964
8,-122.507667,-86.909366
